# Log data  from Hubway for 24 h, 1 Hz

6 Oct 2015

In [2]:
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = (8,6)

import contextlib
import urllib
from lxml import etree
import datetime
import pandas as pd
import time
import os

In [3]:
def getupdate():
    u = 'http://www.thehubway.com/data/stations/bikeStations.xml'
    # tweak of urlopen from hubway_repeatingquery, see http://stackoverflow.com/a/1522709
    with contextlib.closing(urllib.urlopen(u)) as x:
        data = etree.parse(x)
    stations = data.findall('station')
    everything = [[elt.text for elt in station.getchildren()] for station in stations]
    df = pd.DataFrame(everything, columns = [elt.tag for elt in data.find('station')]).convert_objects(convert_numeric=True)
    df.set_index('name', inplace=True)
    return df

In [9]:
def logdata(q1, q2, interval, numpts):
    df = getupdate()
    dflog1 = pd.DataFrame(df[q1])
    dflog1.columns = [datetime.datetime.today()]
    dflog2 = pd.DataFrame(df[q2])
    dflog2.columns = [datetime.datetime.today()]
    i = 1
    while i < numpts:
        time.sleep(interval)
        df = getupdate()
        dflog1[datetime.datetime.today()] = df[q1]
        dflog2[datetime.datetime.today()] = df[q2]
        i = i+1
        if i % 60 == 0:
            print 'completed {} of {}'.format(i, numpts)
    return dflog1.transpose(), dflog2.transpose()

In [ ]:
# let's let it run for a 24 hour at 1 Hz...
# start around midnight to get nice diurnal cycle
time.sleep(60*60*6)
bikelog, docklog = logdata('nbBikes', 'nbEmptyDocks', 60, 60*60*24)

In [ ]:
bikelog.head()

## Export logdata to CSV

In [ ]:
bikelog.to_csv('bikelog_100715.csv')
docklog.to_csv('docklog_100715.csv')